In [1]:
from seqlearn.hmm import MultinomialHMM
import pandas as pd
from pathlib import Path
import numpy as np
from scipy.special import logsumexp
from scipy.sparse import csr_matrix

DATA = Path('../data')
TARGET = 'iob_ner'

In [22]:
def count_transitions(y):
    num_state = np.unique(y).shape[0]
    result = np.zeros((num_state, num_state), dtype=np.uint32)
    for i in range(y.shape[0] - 1):
        result[y[i], y[i + 1]] += 1
    return result


def compute_transition_probs(trans_counts, smoothing=None, alpha=None):
    if smoothing == 'add' and alpha is not None:        
        trans_counts = trans_counts + alpha
    div = trans_counts.sum(axis=0)
    return np.divide(trans_counts, div, where=div!=0)


def check_X_ohe(X):
    if X.min() != 0 or X.max() != 1 or X.unique().shape[0] != 2:
        raise Exception('All features in X should be one hor encoded')


def compute_emission_probs_ohe(X, y, smoothing=None, alpha=0.01):
#     check_X_ohe(X)

    classes, y = np.unique(y, return_inverse=True)
    Y = y.reshape(-1, 1) == np.arange(len(classes))
    counts = Y.T @ X
    if smoothing == 'add' and alpha is not None:
        counts = counts + alpha
    div = counts.sum(axis=0)
    return np.divide(counts, div, where=div != 0)


def compute_emission_probs(X, y, smoothing=None, alpha=None):
    num_features = np.unique(X).shape[0]
    num_classes = np.unique(y).shape[0]
    result = np.zeros((num_classes, num_features))
    for word, tag in zip(X, y):
        result[tag, word] += 1
    if smoothing == 'add' and alpha is not None:
        result = result + alpha
    div = result.sum(axis=0)
    return np.divide(result, div, where=div != 0)


def compute_init_probs(y):
    _, counts = np.unique(y, return_counts=True)
    return counts / y.shape[0]

In [3]:
def get_X_y_lengths(df: pd.DataFrame, cols_to_keep=None, sequence_column='seq', target=TARGET, one_hot=False):
    if isinstance(sequence_column, str):
        sequence_column = [sequence_column]
    if cols_to_keep is None:
        cols_to_keep = {}
        
    y = df[target].cat.codes.values.copy()
    lengths = df.groupby(sequence_column, sort=False).count().iloc[:, 0].values
    if target in cols_to_keep:
        cols_to_keep.remove(target)
    cols_to_drop = set(df.columns) - cols_to_keep
    X = df.drop(cols_to_drop, axis=1)
    if one_hot:
        X = pd.get_dummies(X, dtype=np.bool, sparse=True)
    else:
        X = X.values if X.shape[1] > 1 else np.ravel(X.values)

    return X, y, lengths

In [4]:
train = pd.read_csv(DATA / 'basic_processing.csv', nrows=10000)
train['seq'] = train.groupby(['document', 'part']).grouper.group_info[0]
train = train.astype('category')
train.head()

,token,pos,lemma,iob_ner,part,document,sentence,shape,stem,seq
0,Thousands,NNS,thousand,O,p00,d0018,0,Xxxxxxxxx,thousand,1
1,demonstrators,NNS,demonstrator,O,p00,d0018,0,xxxxxxxxxxxxx,demonstr,1
2,marched,VBN,march,O,p00,d0018,0,xxxxxxx,march,1
3,London,NNP,london,B-geo,p00,d0018,0,Xxxxxx,london,1
4,protest,VB,protest,O,p00,d0018,0,xxxxxxx,protest,1


In [71]:
X, y, lengths = get_X_y_lengths(train, cols_to_keep={'token'}, one_hot=True)

In [72]:
y_str = np.asarray(pd.Categorical.from_codes(y, train.iob_ner.cat.categories), dtype=str)

In [73]:
seqhmm = MultinomialHMM()
seqhmm.fit(X, y_str, lengths=[X.shape[0]])

MultinomialHMM(alpha=0.01, decode='viterbi')

In [ ]:
X_label, _, _ = get_X_y_lengths(train, cols_to_keep={'token'}, one_hot=False)
X_idx = pd.Categorical(X_label).codes

In [70]:
np.log(compute_emission_probs(X_idx, y, smoothing='add', alpha=0.01))

array([[-9.675143  , -4.76217393, -6.03308622, ..., -4.76217393,
        -4.76217393, -4.76217393],
       [-9.675143  , -4.76217393, -6.03308622, ..., -4.76217393,
        -4.76217393, -4.76217393],
       [-9.675143  , -4.76217393, -6.03308622, ..., -4.76217393,
        -4.76217393, -4.76217393],
       ...,
       [-9.675143  , -4.76217393, -6.03308622, ..., -4.76217393,
        -4.76217393, -4.76217393],
       [-9.675143  , -4.76217393, -1.4179657 , ..., -4.76217393,
        -4.76217393, -4.76217393],
       [-0.02005271, -0.14705342, -0.32597596, ..., -0.14705342,
        -0.14705342, -0.14705342]])

In [62]:
X = X.astype(np.uint8)
Y = Y.astype(np.uint8)

In [66]:
logf = np.log(Y.T @ X + 0.01)
logf -= logsumexp(logf, axis=0)

In [67]:
logf

array([[-9.675143  , -4.76217393, -6.03308622, ..., -4.76217393,
        -4.76217393, -4.76217393],
       [-9.675143  , -4.76217393, -6.03308622, ..., -4.76217393,
        -4.76217393, -4.76217393],
       [-9.675143  , -4.76217393, -6.03308622, ..., -4.76217393,
        -4.76217393, -4.76217393],
       ...,
       [-9.675143  , -4.76217393, -6.03308622, ..., -4.76217393,
        -4.76217393, -4.76217393],
       [-9.675143  , -4.76217393, -1.4179657 , ..., -4.76217393,
        -4.76217393, -4.76217393],
       [-0.02005271, -0.14705342, -0.32597596, ..., -0.14705342,
        -0.14705342, -0.14705342]])

In [76]:
seqhmm.coef_

array([[-5.37989735, -4.76217393, -5.37989735, ..., -4.76217393,
        -4.76217393, -4.76217393],
       [-5.37989735, -4.76217393, -5.37989735, ..., -4.76217393,
        -4.76217393, -4.76217393],
       [-5.37989735, -4.76217393, -5.37989735, ..., -4.76217393,
        -4.76217393, -4.76217393],
       ...,
       [-5.37989735, -4.76217393, -5.37989735, ..., -4.76217393,
        -4.76217393, -4.76217393],
       [-5.37989735, -4.76217393, -0.76477684, ..., -4.76217393,
        -4.76217393, -4.76217393],
       [-0.76477684, -0.14705342, -0.76477684, ..., -0.14705342,
        -0.14705342, -0.14705342]])

In [91]:
lengths = np.array([int(len(y) * 0.2), int(len(y) * 0.8)])
lengths = np.array([len(y)])

In [94]:
end = np.cumsum(lengths)
start = end - lengths
start

array([0])

In [95]:
Y[start]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=uint8)

In [130]:
x_test = np.random.randint(0, 5, 10)
y_test = np.random.randint(0, 3, 10)

In [138]:
x_test

array([4, 2, 1, 2, 0, 0, 1, 4, 3, 0])

In [131]:
X_test = x_test.reshape(-1, 1) == np.arange(5)
X_test

array([[False, False, False, False,  True],
       [False, False,  True, False, False],
       [False,  True, False, False, False],
       [False, False,  True, False, False],
       [ True, False, False, False, False],
       [ True, False, False, False, False],
       [False,  True, False, False, False],
       [False, False, False, False,  True],
       [False, False, False,  True, False],
       [ True, False, False, False, False]])

In [132]:
y_test

array([2, 1, 0, 1, 1, 2, 2, 2, 0, 1])

In [133]:
hmm_test = MultinomialHMM()
hmm_test.fit(X_test, y_test, [len(y_test)])

MultinomialHMM(alpha=0.01, decode='viterbi')

In [134]:
np.exp(hmm_test.coef_)

array([[0.00492611, 0.49753695, 0.00970874, 0.98058252, 0.00970874],
       [0.49753695, 0.00492611, 0.98058252, 0.00970874, 0.00970874],
       [0.49753695, 0.49753695, 0.00970874, 0.00970874, 0.98058252]])

In [135]:
np.exp(hmm_test.intercept_trans_)

array([[0.00492611, 0.49875931, 0.00330033],
       [0.49753695, 0.25062035, 0.33333333],
       [0.49753695, 0.25062035, 0.66336634]])

In [136]:
np.exp(hmm_test.intercept_init_)

array([0.00970874, 0.00970874, 0.98058252])

In [137]:
XT = np.array([0, 1, 2, 3]).reshape(-1, 1) == np.arange(5)
hmm_test.predict(XT)

array([2, 0, 1, 0])

In [127]:
(np.unique(XT) == np.array([1, 0])).all()

False

In [129]:
XT.min() == 0

True